<a href="https://colab.research.google.com/github/OmdenaAI/omdena-colombia-socialnetwork/blob/task-5-visualization/src/tasks/task-5-visualization/Carlos_extract_colombian_locations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
def clean_text(text):
  text = text.lower()
  text = text.replace("á","a")
  text = text.replace("à", "a")
  text = text.replace("é","e")
  text = text.replace("í","i")
  text = text.replace("ó","o")
  text = text.replace("ú","u")
  text = text.replace("ü","u")
  text = text.strip()
  text = text.replace("🇨🇴","")
  text = text.replace(",d.c","")
  text = text.replace("d.c","")
  text = text.replace(", d.c","")
  text = text.replace("."," ")
  text = text.replace("&","")
  text = text.replace("#","")
  text = text.replace("-",",")
  text = text.replace("/",",")
  #text = text.replace("polombia","colombia")
  #text = text.replace("colombia","")
  text = text.replace(" dc","")
  text = text.replace("@","")
  #text = text.replace("¬", " ")
  return text

In [ ]:
df = pd.read_csv("User_Location Dictionary.csv", index_col="Unnamed: 0")
df.dropna(how="all", subset=["Location"],inplace=True)
df.head()

,User,Location
2,DS_NAN,BS AS ARGENTINA
3,Robertob_ad,venezuela
5,julio2j,Venezuela
8,07mike,&
10,josealejandrou,"Maracaibo, Venezuela"


In [ ]:
df.shape

(6559, 2)

In [ ]:
df["cleaned_location"] = df["Location"].apply(clean_text)
df = df[df["cleaned_location"] != ""]
df.head()

,User,Location,cleaned_location
2,DS_NAN,BS AS ARGENTINA,bs as argentina
3,Robertob_ad,venezuela,venezuela
5,julio2j,Venezuela,venezuela
10,josealejandrou,"Maracaibo, Venezuela","maracaibo, venezuela"
12,magsegovia,"Lima, Peru","lima, peru"


In [ ]:
df.shape

(6550, 3)

In [ ]:
def is_not_colombian(cleaned_location):
  countries = ["argentina", "venezuela", "peru", "chile", "spain", 
               "españa", "france", "costa rica", "uruguay", "cuba",
               "italia", "england", "ecuador", "el salvador", "nicaragua",
               "italy", "mexico", "rusia"]
  ans = False 
  for country in countries: 
    if country in cleaned_location: 
      ans = True
    else: 
      ans = ans or False
  return ans  

In [ ]:
df["not_colombian"] = df["cleaned_location"].apply(is_not_colombian)
df.head()

,User,Location,cleaned_location,not_colombian
2,DS_NAN,BS AS ARGENTINA,bs as argentina,True
3,Robertob_ad,venezuela,venezuela,True
5,julio2j,Venezuela,venezuela,True
10,josealejandrou,"Maracaibo, Venezuela","maracaibo, venezuela",True
12,magsegovia,"Lima, Peru","lima, peru",True


In [ ]:
df = df[df["not_colombian"] == False]

In [ ]:
df.shape

(5867, 4)

In [ ]:
colombian_locations = pd.read_csv("table-1.csv",usecols=["City", "Department"])
colombian_locations["city_preprocessed"] = colombian_locations["City"].apply(clean_text)
colombian_locations["department_preprocessed"] = colombian_locations["Department"].apply(clean_text)

colombian_dict_cities = {}
colombian_dict_departments = {}

for _, row in colombian_locations.iterrows():
  colombian_dict_cities[row["city_preprocessed"]] = row["department_preprocessed"]
  colombian_dict_departments[row["department_preprocessed"]] = row["city_preprocessed"]

colombian_dict_cities["santiago de cali"] = colombian_dict_cities["cali"]
colombian_locations.head()

,City,Department,city_preprocessed,department_preprocessed
0,Bogotá,Capital District,bogota,capital district
1,Leticia,Amazonas,leticia,amazonas
2,El Encanto,Amazonas,el encanto,amazonas
3,La Chorrera,Amazonas,la chorrera,amazonas
4,La Pedrera,Amazonas,la pedrera,amazonas


In [ ]:
df["location_array"] = df["cleaned_location"].str.split(",")
df.head()

,User,Location,cleaned_location,not_colombian,location_array
15,Cephalosporium,"Cali, Colombia","cali, colombia",False,"[cali, colombia]"
16,Lucio_Biglione,"#General Cabrera,Cordoba","general cabrera,cordoba",False,"[general cabrera, cordoba]"
21,FTG84,Corvera d'Asturies,corvera d'asturies,False,[corvera d'asturies]
27,arielucho1970,Ciudad Autónoma de Buenos Aire,ciudad autonoma de buenos aire,False,[ciudad autonoma de buenos aire]
28,Camiloramirez_s,"Bogotá , Colombia","bogota , colombia",False,"[bogota , colombia]"


In [ ]:
def get_complete_location_from_csv(location_array):
  city = ""
  department = ""
  country = ""
  for location_item in location_array: 
    location_item = location_item.strip()
    location_item = location_item.replace(" colombia", "")
    location_item = location_item.replace("colombia ", "")
    if location_item == "colombia" and len(location_array) == 1:
      city = "bogota"
      department = "capital district"
      country = "colombia"
    elif colombian_dict_cities.get(location_item):
      city = location_item
      department = colombian_dict_cities.get(location_item)
      country = "colombia"
      break
    elif colombian_dict_departments.get(location_item):
      city = colombian_dict_departments.get(location_item)
      department = location_item 
      country = "colombia"
  return pd.Series([city,department,country])

In [ ]:
df[["city", "department", "country"]] = df["location_array"].apply(get_complete_location_from_csv)

In [ ]:
df[df["country"] == "colombia"]

,User,Location,cleaned_location,not_colombian,location_array,city,department,country
15,Cephalosporium,"Cali, Colombia","cali, colombia",False,"[cali, colombia]",cali,valle del cauca,colombia
16,Lucio_Biglione,"#General Cabrera,Cordoba","general cabrera,cordoba",False,"[general cabrera, cordoba]",cordoba,quindio,colombia
28,Camiloramirez_s,"Bogotá , Colombia","bogota , colombia",False,"[bogota , colombia]",bogota,capital district,colombia
33,RichardRiCampo,Cali - Valle,"cali , valle",False,"[cali , valle]",cali,valle del cauca,colombia
38,activistica,Colombia,colombia,False,[colombia],bogota,capital district,colombia
...,...,...,...,...,...,...,...,...
10535,G_abrielaaa_,"Bogotá, D.C., Colombia","bogota, , colombia",False,"[bogota, , colombia]",bogota,capital district,colombia
10536,jennypaosuarezg,BOGOTA - COLOMBIA,"bogota , colombia",False,"[bogota , colombia]",bogota,capital district,colombia
10537,Martinpz02,"Bogotá, D.C., Colombia","bogota, , colombia",False,"[bogota, , colombia]",bogota,capital district,colombia
10539,somosmestizzo,bogota,bogota,False,[bogota],bogota,capital district,colombia


In [ ]:
df[df["country"] != "colombia"]

,User,Location,cleaned_location,not_colombian,location_array,city,department,country
21,FTG84,Corvera d'Asturies,corvera d'asturies,False,[corvera d'asturies],,,
27,arielucho1970,Ciudad Autónoma de Buenos Aire,ciudad autonoma de buenos aire,False,[ciudad autonoma de buenos aire],,,
35,ConvocatoriasGz,Galiza,galiza,False,[galiza],,,
37,williamgarzon86,http://williamgarzon86,"http:,,williamgarzon86",False,"[http:, , williamgarzon86]",,,
40,tecnochamana,Theremín.,theremin,False,[theremin ],,,
...,...,...,...,...,...,...,...,...
10435,AndresParbe,Deportivo Cali ois.,deportivo cali ois,False,[deportivo cali ois ],,,
10471,JotaAEmeA,Κολομβία,κολομβία,False,[κολομβία],,,
10477,fevelasquezd,narcoestado asesino colombiano,narcoestado asesino colombiano,False,[narcoestado asesino colombiano],,,
10484,emmanuelrangel6,Bogota Dc. Colombia,bogota colombia,False,[bogota colombia],,,


In [ ]:
df[df["country"] == "colombia"].to_csv("cleaned_colombian_locations.csv")